**Install necessary packages**

In [ ]:
!pip install ultralytics
!pip install deep-sort-realtime

import cv2
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO


**Detecting Reverse Vehicle moving on wrong direction**

In [ ]:
# Install necessary packages
# !pip install ultralytics
# !pip install deep-sort-realtime

# import cv2
# import torch
# from deep_sort_realtime.deepsort_tracker import DeepSort
# from ultralytics import YOLO

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')
model.to(device)  # Move the model to the device (GPU/CPU)

# Initialize DeepSORT for tracking
deepsort = DeepSort(max_age=30, n_init=3)

# Define the paths for input and output videos
input_video_path = '/content/drive/MyDrive/Colab Notebooks/VDIWD/WS1.mp4'
output_video_path = '/content/drive/MyDrive/Colab Notebooks/VDIWD/Ws107output_video.mp4'

# Initialize video capture
cap = cv2.VideoCapture(input_video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error opening video file: {input_video_path}")
    exit()

# Get video properties for the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define the region of interest (ROI) to be the middle 20% of the frame (10% extra on the right)
roi_x1 = int(frame_width * 0.45)  # Start of middle 10% (45% into the width)
roi_x2 = int(frame_width * 0.65)  # End of middle 20% (65% into the width)

# Initialize a dictionary to store vehicle positions
positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB as YOLOv8 expects RGB images
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform object detection with YOLOv8
    results = model(img)

    # Extract detections for vehicles only (ignoring people)
    detections = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            if cls in [2, 3, 5, 7]:  # Filter for cars (2), motorcycles (3), buses (5), trucks (7)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()

                # Apply the ROI condition to focus on the middle 20% (10% extra on the right)
                if x1 >= roi_x1 and x2 <= roi_x2:
                    detections.append([x1, y1, x2, y2, conf])

    # Correctly format detections for DeepSORT
    detections_for_sort = [[d[:4], d[4]] for d in detections]

    # Update tracker
    tracks = deepsort.update_tracks(detections_for_sort, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = track.to_tlbr().astype(int)
        centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))

        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append(centroid)

        # Analyze the motion
        if len(positions[track_id]) > 2:
            dy = positions[track_id][-1][1] - positions[track_id][-2][1]

            # Focus only on vehicles moving in the reverse direction (upward in the frame)
            if dy < 0:
                color = (0, 0, 255)  # Red for reverse
                label = f"Reverse ID: {track_id}"

                # Draw bounding box for reverse-moving vehicles only
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

    # Optional: Display the frame (disable if running in non-interactive environment)
    # cv2.imshow('frame', frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved to: {output_video_path}")


Using device: cpu

0: 384x640 9 persons, 16 cars, 386.8ms
Speed: 3.5ms preprocess, 386.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 16 cars, 391.0ms
Speed: 3.5ms preprocess, 391.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 16 cars, 1 motorcycle, 397.6ms
Speed: 5.5ms preprocess, 397.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 16 cars, 1 motorcycle, 401.5ms
Speed: 4.7ms preprocess, 401.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 16 cars, 1 motorcycle, 476.5ms
Speed: 3.6ms preprocess, 476.5ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 15 cars, 474.1ms
Speed: 3.9ms preprocess, 474.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 14 cars, 468.3ms
Speed: 9.9ms preprocess, 468.3ms inference, 2.8ms postprocess per imag

**Detecting Reverse Vehicle moving on wrong direction Demo 2**

In [ ]:
# Install necessary packages
!pip install ultralytics
!pip install deep-sort-realtime

import cv2
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')
model.to(device)  # Move the model to the device (GPU/CPU)

# Initialize DeepSORT for tracking
deepsort = DeepSort(max_age=30, n_init=3)

# Define the paths for input and output videos
input_video_path = '/content/drive/MyDrive/Colab Notebooks/VDIWD/WS2.mp4'
output_video_path = '/content/drive/MyDrive/Colab Notebooks/VDIWD/Ws207output_video.mp4'

# Initialize video capture
cap = cv2.VideoCapture(input_video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error opening video file: {input_video_path}")
    exit()

# Get video properties for the output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define the region of interest (ROI) to be the middle 20% of the frame (10% extra on the right)
roi_x1 = int(frame_width * 0.45)  # Start of middle 10% (45% into the width)
roi_x2 = int(frame_width * 0.65)  # End of middle 20% (65% into the width)

# Initialize a dictionary to store vehicle positions
positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB as YOLOv8 expects RGB images
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform object detection with YOLOv8
    results = model(img)

    # Extract detections for vehicles only (ignoring people)
    detections = []
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            if cls in [2, 3, 5, 7]:  # Filter for cars (2), motorcycles (3), buses (5), trucks (7)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf[0].item()

                # Apply the ROI condition to focus on the middle 20% (10% extra on the right)
                if x1 >= roi_x1 and x2 <= roi_x2:
                    detections.append([x1, y1, x2, y2, conf])

    # Correctly format detections for DeepSORT
    detections_for_sort = [[d[:4], d[4]] for d in detections]

    # Update tracker
    tracks = deepsort.update_tracks(detections_for_sort, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = track.to_tlbr().astype(int)
        centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))

        if track_id not in positions:
            positions[track_id] = []
        positions[track_id].append(centroid)

        # Analyze the motion
        if len(positions[track_id]) > 2:
            dy = positions[track_id][-1][1] - positions[track_id][-2][1]

            # Focus only on vehicles moving in the reverse direction (upward in the frame)
            if dy < 0:
                color = (0, 0, 255)  # Red for reverse
                label = f"Reverse ID: {track_id}"

                # Draw bounding box for reverse-moving vehicles only
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

    # Optional: Display the frame (disable if running in non-interactive environment)
    # cv2.imshow('frame', frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Output video saved to: {output_video_path}")


Streaming output truncated to the last 5000 lines.
Speed: 5.9ms preprocess, 394.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 16 cars, 3 buss, 358.8ms
Speed: 3.2ms preprocess, 358.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 15 cars, 4 buss, 350.4ms
Speed: 7.6ms preprocess, 350.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 14 cars, 3 buss, 381.5ms
Speed: 3.7ms preprocess, 381.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 15 cars, 5 buss, 1 truck, 348.9ms
Speed: 9.9ms preprocess, 348.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 15 cars, 4 buss, 1 truck, 392.9ms
Speed: 3.5ms preprocess, 392.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 14 cars, 1 motorcycle, 4 buss, 1 truck, 356.8ms
Speed: 3.9ms preprocess, 356.8ms